In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import data

weeks = range(1, 19)
before_dfs = []
after_dfs = []

for week in weeks:
    week_str = f"{week:02d}"  # Zero-padded week string
    before_file = f"nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/train/input_2023_w{week_str}.csv"
    after_file  = f"nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/train/output_2023_w{week_str}.csv"
    before_dfs.append(pd.read_csv(before_file))
    after_dfs.append(pd.read_csv(after_file))

before_pass = pd.concat(before_dfs, ignore_index=True)
after_pass = pd.concat(after_dfs, ignore_index=True)
supp = pd.read_csv('nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/supplementary_data.csv')


C:\Users\cools\AppData\Local\Temp\ipykernel_29956\1834361638.py:16: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supp = pd.read_csv('nfl-big-data-bowl-2026-analytics/114239_nfl_competition_files_published_analytics_final/supplementary_data.csv')


In [3]:
# Find max frame for each play (right as the pass is thrown), filter by receiver, filter by speed/accel, and remove GO routes
max_frames = (
    before_pass.groupby(['game_id', 'play_id'], as_index=False)['frame_id']
    .max()
    .rename(columns={'frame_id': 'max_frame_id'})
)
max_frames = max_frames[max_frames['max_frame_id'] <=30]
receiver_f1 = before_pass[before_pass['player_role'] == 'Targeted Receiver']
receiver_f2 = receiver_f1.merge(max_frames, left_on = ['game_id', 'play_id', 'frame_id'], right_on =['game_id', 'play_id', 'max_frame_id'], how = 'inner')
receiver_supp = receiver_f2.merge(supp, on = ['game_id', 'play_id'])

receiver_supp = receiver_supp[(receiver_supp['s'] >= 5) | (receiver_supp['a'] >= 5) ]
receiver_supp = receiver_supp[receiver_supp['route_of_targeted_receiver'] != 'GO']

In [287]:


FPS = 10          # frames per second (NGS pre/early is 10 Hz)
RUN_LEN = 5       # need 5 straight frames
THETA = 35.0      # deg change vs release heading

def ang_deg(dx, dy):
    return (np.degrees(np.arctan2(dy, dx)) + 360) % 360

def ang_diff(a, b):
    return abs((a - b + 180) % 360 - 180)  # 0..180 smallest diff

# 1) keep only the targeted receiver in after_pass
keys = ["game_id", "play_id", "nfl_id"]
ap = after_pass.merge(receiver_supp[keys].drop_duplicates(), on=keys, how="inner")

# 2) heading per frame (from frame t-1 -> t) for the WR
ap = ap.sort_values(keys + ["frame_id"]).copy()
ap["dx"] = ap.groupby(keys)["x"].diff()
ap["dy"] = ap.groupby(keys)["y"].diff()
ap["heading_deg"] = ang_deg(ap["dx"], ap["dy"])

# 3) heading at release: vector from release point -> first after-pass frame
first_ap = ap.groupby(keys, as_index=False).first()
rel_vec = first_ap.merge(receiver_supp, on=keys, suffixes=("_ap", "_rel"))
rel_vec["heading_rel_deg"] = ang_deg(rel_vec["x_ap"] - rel_vec["x_rel"],
                                     rel_vec["y_ap"] - rel_vec["y_rel"])
rel_head = rel_vec[keys + ["heading_rel_deg", "frame_id_ap"]].rename(columns={"frame_id_ap":"first_after_frame"})

# 4) join release heading back to all after-pass frames
ap = ap.merge(rel_head, on=keys, how="left")
ap["hdiff"] = ang_diff(ap["heading_deg"], ap["heading_rel_deg"])

# 5) find first run of RUN_LEN consecutive frames with hdiff >= THETA
ap["is_cutlike"] = ap["hdiff"] >= THETA
ap["run_sum"] = (
    ap.groupby(keys)["is_cutlike"]
      .apply(lambda s: s.rolling(RUN_LEN, min_periods=RUN_LEN).sum())
      .reset_index(level=keys, drop=True)
)

# take the first frame where the window sum hits RUN_LEN
cut_rows = (
    ap.loc[ap["run_sum"] == RUN_LEN]
      .groupby(keys, as_index=False)
      .first()[keys + ["frame_id", "first_after_frame", "heading_rel_deg"]]
      .rename(columns={"frame_id":"t_cut_frame"})
)

# 6) assemble per-play result
res = receiver_supp[keys].drop_duplicates().merge(cut_rows, on=keys, how="inner")
res["is_timing"] = res["t_cut_frame"].notna()
res["time_to_cut_s"] = (res["t_cut_frame"] - res["first_after_frame"]) / FPS
res = res.drop(columns=["first_after_frame"])

# columns: game_id, play_id, nfl_id, is_timing, t_cut_frame, time_to_cut_s, heading_rel_deg
